In [2]:
from tqdm import tqdm
from functools import reduce

with open("./data_inputs/day11_input.txt") as f:
    input_raw = f.read()

In [3]:
ops = {"+": lambda x: lambda old: old + x,
        "-": lambda x: lambda old: old - x,
        "*": lambda x: lambda old: int(old * x),
        "/": lambda x: lambda old: old // x,
        "**2": lambda : lambda old: old * old,
}

def init_monkeys(input_raw):
    monkeys = {}
    curr_m = 0
    for i, l in enumerate(input_raw.split("\n")):
        j = i % 7
        match j:
            case 0: 
                curr_m = l[7]
                monkeys[curr_m] = {}
            case 1:
                monkeys[curr_m]["items"] = list(map(int, l.split(": ")[1].split(", ")))
                monkeys[curr_m]["inspected_items"] = 0 
            case 2: 
                operation = l.split("old ")[1][0]
                const = l.split(" ")[-1]
                if const == "old": 
                    monkeys[curr_m]["operation"] = ops["**2"]()
                else: 
                    monkeys[curr_m]["operation"] = ops[operation](int(const))
            case 3:
                monkeys[curr_m]["div"] = int(l.split("by ")[1])
            case 4:
                monkeys[curr_m]["throw_true"] = l.split(" ")[-1]
            case 5:
                monkeys[curr_m]["throw_false"] = l.split(" ")[-1]
            case 6:
                pass

    return monkeys

In [4]:
# ---- Part 1 ----

monkeys = init_monkeys(input_raw)

for round in tqdm(range(20)):
    for m in monkeys:
        monkeys[m]["inspected_items"] += len(monkeys[m]["items"])
        for item in monkeys[m]["items"]:
            worry_lvl = monkeys[m]["operation"](item) // 3
            is_test = (worry_lvl % monkeys[m]["div"] == 0)
            throw_to_m = monkeys[m]["throw_true"] if is_test else monkeys[m]["throw_false"]
            monkeys[throw_to_m]["items"].append(worry_lvl) 

        monkeys[m]["items"] = []

inspected_items_list = sorted([monkeys[m]["inspected_items"] for m in monkeys], reverse=True)

print("Result 1:", inspected_items_list[0] * inspected_items_list[1])

100%|██████████| 20/20 [00:00<00:00, 19996.68it/s]

Result 1: 110264


In [5]:
# ---- Part 2 ----

monkeys = init_monkeys(input_raw)
div_primes = [monkeys[m]["div"] for m in monkeys]
primes_prod = reduce((lambda x, y: x * y), div_primes)
# We are only interested in knowing if every number will be divisible by the any of the div numbers from 
# each monkey, so now, as the numbers get incredibly large, we can divide them by the product of all div numbers
# which are prime numbers and only keep the reminder which will be divisible or not in the same way by any of the divs 

for round in tqdm(range(10000)):
    for m in monkeys:
        monkeys[m]["inspected_items"] += len(monkeys[m]["items"])
        for item in monkeys[m]["items"]:
            worry_lvl = monkeys[m]["operation"](item)
            is_test = (worry_lvl % monkeys[m]["div"] == 0)
            throw_to_m = monkeys[m]["throw_true"] if is_test else monkeys[m]["throw_false"]
            # Extracting the reminder of every number by the product of the div prime numbers to keep them small
            monkeys[throw_to_m]["items"].append(worry_lvl % primes_prod)  
            
        monkeys[m]["items"] = []

inspected_items_list = sorted([monkeys[m]["inspected_items"] for m in monkeys], reverse=True)

print("Result 2:", inspected_items_list[0] * inspected_items_list[1])

100%|██████████| 10000/10000 [00:00<00:00, 41244.44it/s]

Result 2: 23612457316
